<h1>Ejemplo de un prestamo</h1>

En este caso analizaremos los datos de un conjunto de operaciones de prestamo.
Cada fila de datos posee el Id de prestamo y si se efectuo o no.

Como primer paso sera capturar los datos a traves de Pandas.
Teniendo 2 opciones, desde un directorio local o desde un sitio web usando el modulo Requests.

Tambien usaremos Matplotlib para la realizacion de graficos.

In [ ]:
import io
import requests
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline


#leyendo desde una ruta fisica
#df = pd.read_csv("D:/Yo/code/ADA/Python/Workshop/Prestamo/data/train_es.csv") #Reading the dataset in a dataframe using Pandas

#leyendo desde una ulr
url="https://shadowdrone.azurewebsites.net/ada/workshop/python/prestamo/data/train_es.csv"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))

Vemos la info de los primeras N filas del dataframe

In [ ]:
df.head(10)

Como siguiente paso, analizamos la distribucion de los items y la distribucion por quartiles.

In [ ]:
df.describe()

Vemos la distribucion por area

In [ ]:
df['Area'].value_counts()

Generamos un historigrama de los ingresos. En este caso vemos como hay outliners que representan algunos ingresos muy altos.

In [ ]:
df['Ingresos'].hist(bins=50)

Generamos los graficos potblox para analizar graficamente la distribucion

In [ ]:
df.boxplot(column='Ingresos')

In [ ]:
df.boxplot(column='Ingresos', by = 'Educacion')

Historigrama y boxplot del importe del prestamo

In [ ]:
df['Importe_Prestamo'].hist(bins=50)

In [ ]:
df.boxplot(column='Importe_Prestamo')

Aquí generamos unos graficos estadisticos para comprender probabilidades de credito usando tablas pivotantes.

In [ ]:
temp1 = df['Historial_Credito'].value_counts(ascending=True)
temp2 = df.pivot_table(values='Estado_Prestamo',index=['Historial_Credito'],aggfunc=lambda x: x.map({'S':1,'N':0}).mean())
print ('Tabla de Frecuencias para el Historial de Credito:') 
print (temp1)

print ('\nProbabilidad de obtener credito para cada clase de Historial de Credito:')
print (temp2)





------------------------
Ahora genero graficos

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,6))
ax1 = fig.add_subplot(121) #121
ax1.set_xlabel('Historial_Credito')
ax1.set_ylabel('Cantidad de Clientes')
ax1.set_title("Clientes por Historial de Credito")
temp1.plot(kind='bar')

ax2 = fig.add_subplot(122)
temp2.plot(kind = 'bar')
ax2.set_xlabel('Historial_Credito')
ax2.set_ylabel('Probabilidad de obtener el prestamo')
ax2.set_title("Probabilidad de obtener el prestamo por historial de credito")

Mas graficos en: https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.figure.html

En este caso hacemos un crosstab del historial de credito por prestamo

In [ ]:
temp3 = pd.crosstab(df['Historial_Credito'], df['Estado_Prestamo'])
temp3.plot(kind='bar', stacked=True, color=['red','blue'], grid=False)

In [ ]:
temp4 = pd.crosstab([df['Historial_Credito'],df['Genero']], df['Estado_Prestamo' ])
temp4.plot(kind='bar', stacked=True, color=['red','blue', 'green', 'orange'], grid=False)

Ya teniendo un panorama de los datos , podremos ver si nos hacen falta llenar datos faltantes y como los llenaremos, y ejecutando diferentes escenarios deacuerdo a las necesidades. Esto se lo conoce como Data munging

In [ ]:
df.apply(lambda x: sum(x.isnull()),axis=0) 

<h1>Clasificación usando ML, pero primero tenemos que llenar los datos faltantes</h1>

In [ ]:
df['Autonomo'].fillna('No',inplace=True)
df['Genero'].fillna(df['Genero'].mode()[0], inplace=True)
df['Casado'].fillna(df['Casado'].mode()[0], inplace=True)
df['Dependientes'].fillna(df['Dependientes'].mode()[0], inplace=True)
df['Dias_Prestamo'].fillna(df['Dias_Prestamo'].mode()[0], inplace=True)
df['Historial_Credito'].fillna(df['Historial_Credito'].mode()[0], inplace=True)

In [ ]:
table = df.pivot_table(values='Importe_Prestamo', index='Autonomo' ,columns='Educacion', aggfunc=np.median)
# Define function to return value of this pivot_table
def fage(x):
 return table.loc[x['Autonomo'],x['Educacion']]
# Replace missing values
df['Importe_Prestamo'].fillna(df[df['Importe_Prestamo'].isnull()].apply(fage, axis=1), inplace=True)

Logaritmo para opacar a los extremos(outliers)

In [ ]:
 df['Importe_Prestamo_log'] = np.log(df['Importe_Prestamo'])
df['Importe_Prestamo_log'].hist(bins=20)

In [ ]:
df['TotalIngresos'] = df['Ingresos'] + df['CoIngresos']
df['TotalIngresos_log'] = np.log(df['TotalIngresos'])
df['TotalIngresos_log'].hist(bins=20) 

Antes de comenzar, necesitammos transformar aquellos datos del dataframe a numero, porque el sklearn trabaja sobre valores numericos, entonces el LabelEncoder lo que hara es generar una representacion numerica de esos valores escritos en letras.

In [ ]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Genero','Casado','Dependientes','Educacion','Autonomo','Area','Estado_Prestamo']
le = LabelEncoder()
for i in var_mod:
    #print(df[i])
    df[i] = le.fit_transform(df[i])
df.dtypes

<h2>Algoritmo de Clasificacion Generico usando CrossValidation</h2>

In [ ]:
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
#from sklearn.cross_validation import KFold   #For K-fold cross validation
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

#Generic function for making a classification model and accessing performance:
def classification_model(model, data, predictors, outcome):
  #Fit the model:
  model.fit(data[predictors],data[outcome])
  
  #Make predictions on training set:
  predictions = model.predict(data[predictors])
  
  #Print accuracy
  accuracy = metrics.accuracy_score(predictions,data[outcome])
  print ("Accuracy : %s" % "{0:.3%}".format(accuracy))

  #Perform k-fold cross-validation with 5 folds
  kf = KFold(n_splits=5) 
  #kf.get_n_splits(data.shape[0])
  error = []
  for train, test in kf.split(data):
    # Filter training data
    train_predictors = (data[predictors].iloc[train,:])
    
    # The target we're using to train the algorithm.
    train_target = data[outcome].iloc[train]
    
    # Training the algorithm using the predictors and target.
    model.fit(train_predictors, train_target)
    
    #Record error from each cross-validation run
    error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
  print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))

  #Fit the model again so that it can be refered outside the function:
  model.fit(data[predictors],data[outcome]) 

<h2>Logistic Regression Model - cuantificamos en forma no lineal las variables a analizar</h2>
<p>
Segun lo analizado antes, podiamos ver que las chances para que puedan aprobarle el credito a alguien puede ser si:

- El solicitante tiene un historial de credito(vimos que habia un porcentaje alto de aquellos que tenian historial y se aprobo un prestamo).
- Los que tengan ingresos altos incluyendo los propios o los de familiares
- Los que tengan nivel alto de educacion
- Otro: areas en donde haya perspectivas de crecimiento
    </p>

Si solo aplicamos solo con la variable 'Historial de Credito'

In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

outcome_var = 'Estado_Prestamo'
model = LogisticRegression()
predictor_var = ['Historial_Credito']
classification_model(model, df,predictor_var,outcome_var)

otra prediccion, pero usando mas variables

In [ ]:
#probamos con otras variables
predictor_var = ['Historial_Credito','Educacion','Casado','Autonomo','Area']
classification_model(model, df,predictor_var,outcome_var)

Testeamos con nuevos valores de un archivo de testeo nuevo(5 registros tiene).

In [ ]:
#leyendo desde una ruta fisica
#dfT1 = pd.read_csv("D:/Yo/code/ADA/Python/Workshop/Prestamo/data/test_es_rgl.csv") #Reading the dataset in a dataframe using Pandas

#leyendo desde una ulr
url="https://shadowdrone.azurewebsites.net/ada/workshop/python/prestamo/data/test_es_rgl.csv"
s=requests.get(url).content
dfT1=pd.read_csv(io.StringIO(s.decode('utf-8')))

dfT1.head(10)

transformando las variables a valor numerico porque el modelo que usamos necesita los valores numericos.

In [ ]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Casado','Educacion','Autonomo','Area']
le = LabelEncoder()
for i in var_mod:
    #print(df[i])
    dfT1[i] = le.fit_transform(dfT1[i])
dfT1.dtypes

Aquí vamos a poder ver a quien le prestariamos dinero.

In [ ]:
#print(dfT1)

predicciones = model.predict(dfT1)
print(predicciones)

<h2>Decision Tree - Otro modelo usando mas variables.</h2>

In [ ]:
model = DecisionTreeClassifier()
predictor_var = ['Historial_Credito','Genero','Casado','Educacion']
classification_model(model, df,predictor_var,outcome_var)

Como da mas o menos lo mismo que lo anterior, ponemos las variables numericas ya que el Historial Crediticio sigue dominando

In [ ]:
predictor_var = ['Historial_Credito','Dias_Prestamo','Importe_Prestamo_log']
classification_model(model, df,predictor_var,outcome_var)

In [ ]:
#leyendo desde una ruta fisica
#dfT2 = pd.read_csv("D:/Yo/code/ADA/Python/Workshop/Prestamo/data/test_es_dt.csv") #Reading the dataset in a dataframe using Pandas

#leyendo desde una ulr
url="https://shadowdrone.azurewebsites.net/ada/workshop/python/prestamo/data/test_es_dt.csv"
s=requests.get(url).content
dfT2=pd.read_csv(io.StringIO(s.decode('utf-8')))

dfT2.head(10)

In [ ]:
predicciones = model.predict(dfT2)
print(predicciones)